# EMAIL AUTOMATION

In [ ]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [3]:
import base64
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def authenticate_gmail():
    """Authenticate with the Gmail API and return the service object."""
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

def create_email(sender, recipient, subject, body):
    """Create an email message."""
    message = MIMEMultipart()
    message['From'] = sender
    message['To'] = recipient
    message['Subject'] = subject

    # Add the email body
    message.attach(MIMEText(body, 'plain'))

    # Encode the message to base64
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw_message}

def send_email_batch(service, sender, recipients, subject, body):
    """Send emails in batch to avoid Gmail API limits and ensure efficiency."""
    batch_size = 100  # Send emails in batches of 100 to avoid rate limits

    for i in range(0, len(recipients), batch_size):
        batch_recipients = recipients[i:i + batch_size]
        for recipient in batch_recipients:
            email_message = create_email(sender, recipient, subject, body)
            try:
                service.users().messages().send(userId='me', body=email_message).execute()
                print(f"Email sent to {recipient}")
            except Exception as e:
                print(f"Failed to send email to {recipient}: {e}")

if __name__ == '__main__':
    # Replace with your details
    sender_email = "gulshankumar02985@gmail.com"
    subject = "EMAIL AUTOMATION "
    email_body = "this is generated and send you from gulshan kumar the upcoming billionare of this country"

    # Load recipient list from a file (e.g., recipients.txt)
    with open('recipients.txt', 'r') as file:
        recipient_emails = [line.strip() for line in file.readlines()]

    # Authenticate and send emails
    gmail_service = authenticate_gmail()
    send_email_batch(gmail_service, sender_email, recipient_emails, subject, email_body)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=637536540109-jqnhfl0jhs67gtn4s9pr5pkc4s7bb9fh.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60079%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=vfJCWjkgGQeUqTCTOXcgRzBL5VNnEU&access_type=offline
Email sent to raushankr2649@gmail.com
Email sent to raushankr4452649@gmail.com
Email sent to gulshankumar02985@gmail.com
Email sent to raushanneet01@gmail.com
Email sent to raushanneet02@gmail.com
Email sent to gulshankumar607979@gmail.com
Email sent to rk264849@gmail.com


# if in mail link, QR CODE ,image is to be provide then 

In [12]:
pip install qrcode[pil]


   ---------------------------------------- 0.0/45.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/45.7 kB ? eta -:--:--
   -------- ------------------------------- 10.2/45.7 kB ? eta -:--:--
   ----------------- ---------------------- 20.5/45.7 kB 330.3 kB/s eta 0:00:01
   ----------------- ---------------------- 20.5/45.7 kB 330.3 kB/s eta 0:00:01
   ---------------------------------------- 45.7/45.7 kB 227.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
import base64
import os
import time
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from io import BytesIO
import qrcode

# Gmail API Scopes
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def authenticate_gmail():
    """Authenticate with the Gmail API and return the service object."""
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

def generate_qr_code(data):
    """Generate a QR code image from the given data."""
    qr = qrcode.make(data)
    img_byte_arr = BytesIO()
    qr.save(img_byte_arr, format='PNG')
    img_byte_arr.seek(0)
    return img_byte_arr.read()

def create_email(sender, recipient, subject, body, image_paths=None, qr_code_url=None):
    """Create an email message with HTML content, QR codes, and images."""
    message = MIMEMultipart('related')
    message['From'] = sender
    message['To'] = recipient
    message['Subject'] = subject

    # HTML body
    html_body = f"""
    <html>
        <body>
            <p>{body}</p>
            <p>Here's a link: <a href="https://www.bing.com/images/search?q=image+of+radha+rani&form=HDRSC3&first=1">Click here</a></p>
            <p>Scan this QR Code below:</p>
            <img src="cid:qr_code_image" alt="QR Code" />
    """

    # Generate and attach QR code
    if qr_code_url:
        qr_img = generate_qr_code(qr_code_url)
        qr_attachment = MIMEImage(qr_img, _subtype='png')
        qr_attachment.add_header('Content-ID', '<qr_code_image>')
        qr_attachment.add_header('Content-Disposition', 'inline', filename='qr_code.png')
        message.attach(qr_attachment)

    # Attach additional images
    if image_paths:
        for i, image_path in enumerate(image_paths):
            with open(image_path, 'rb') as img_file:
                img_data = img_file.read()
                img = MIMEImage(img_data)
                img_cid = f'image{i}'  # Unique Content-ID for each image
                img.add_header('Content-ID', f'<{img_cid}>')
                img.add_header('Content-Disposition', 'inline', filename=os.path.basename(image_path))
                message.attach(img)
                html_body += f'<p>Attached Image:</p><img src="cid:{img_cid}" alt="Attached Image" /><br>'

    # Complete HTML body
    html_body += "</body></html>"
    html_part = MIMEText(html_body, 'html')
    message.attach(html_part)

    # Encode message in base64
    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw_message}

def send_email_batch(service, sender, recipients, subject, body, image_paths=None, qr_code_url=None):
    """Send emails in batches to avoid Gmail limits."""
    batch_size = 450  # Adjust based on Gmail's limits
    total_sent = 0

    for i in range(0, len(recipients), batch_size):
        batch_recipients = recipients[i:i + batch_size]
        print(f"Sending batch {i // batch_size + 1}: {len(batch_recipients)} recipients")
        for recipient in batch_recipients:
            email_message = create_email(sender, recipient, subject, body, image_paths, qr_code_url)
            try:
                service.users().messages().send(userId='me', body=email_message).execute()
                print(f"Email sent to {recipient}")
                total_sent += 1
                time.sleep(1)  # Delay to respect rate limits
            except Exception as e:
                print(f"Failed to send email to {recipient}: {e}")
        print(f"Batch {i // batch_size + 1} completed.")
        time.sleep(5)  # Pause between batches

    print(f"Total emails sent: {total_sent} out of {len(recipients)}")

if __name__ == '__main__':
    # Gmail account details
    sender_email = "your-email@gmail.com"
    subject = "Email with QR Code, Images, and Links"
    email_body = "This is a rich-content email with a QR code, images, and links."

    # Attachments and QR code URL
    image_files = [
        "C:/Users/gulsh/Downloads/IMAGES/wp2765079-lord-krishna-and-arjuna-hd-wallpapers.jpg"
    ]
    qr_code_url = "https://www.bing.com/images/search?view=detailV2&ccid=kYDdGx5Y&id=3E1FF2CCDE1B7FC1A7064DEBF9E2D27E40D99450&thid=OIP.kYDdGx5Y8-P5CFsxWgZkuwHaNK&mediaurl=https%3a%2f%2fi.pinimg.com%2f736x%2f9e%2f8d%2fbe%2f9e8dbe81289ec9f10e50d90cbb81c8db.jpg&exph=1308&expw=736&q=image+of+radha+rani&simid=607991826165946567&FORM=IRPRST&ck=86552C0351CF0373EBE95E2C2A68B9D9&selectedIndex=3&itb=0"

    # Recipient list (load from a file)
    with open('recipients.txt', 'r') as file:
        recipient_emails = [line.strip() for line in file.readlines()]

    # Authenticate and send emails
    gmail_service = authenticate_gmail()
    send_email_batch(gmail_service, sender_email, recipient_emails, subject, email_body, image_files, qr_code_url)


Sending batch 1: 2 recipients
Email sent to gulshankumar02985@gmail.com
Email sent to gulshankumar607979@gmail.com
Batch 1 completed.
Total emails sent: 2 out of 2
